In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%%bigquery --project bi-gojek df3
WITH
date_list AS
(
  SELECT DATE('2020-06-04') ref_date0, 
         DATE('2020-06-14') ref_date1
),

calc_dexguard as(
select
  d1,
  'dexguard' as key,
  count(distinct CONCAT(x_owner_id, " ", x_owner_type)) as conn_account,
  array_agg(distinct CONCAT(x_owner_id, " ", x_owner_type)) as accounts_list
from `staging.fd_mdf_fingerprint_feature_flat` 
where true
and date(updated_time) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
-- exclude empty and nulls dexguard due to coverage issues, also iOS users have null dexguards
and d1 is not null 
and d1 <> ''
-- THIS IS OPTIONAL depending on what type of users you want to get the connected accounts of
group by 1
),

calc_uniqueid as(
select
  x_uniqueid,
  'unique_id' as key,
  count(distinct CONCAT(x_owner_id, " ", x_owner_type)) as conn_account,
  array_agg(distinct CONCAT(x_owner_id, " ", x_owner_type)) as accounts_list
from `staging.fd_mdf_fingerprint_feature_flat` 
where true
and date(updated_time) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
-- exclude empty and nulls
and x_uniqueid is not null
and x_uniqueid <> ''
-- THIS IS OPTIONAL depending on what type of users you want to get the connected accounts of
group by 1
),

calc_serial_number as(
select
  device_serial_number,
  'serial_number' as key,
  count(distinct CONCAT(x_owner_id, " ", x_owner_type)) as conn_account,
  array_agg(distinct CONCAT(x_owner_id, " ", x_owner_type)) as accounts_list
from `staging.fd_mdf_fingerprint_feature_flat` 
where true
and date(updated_time) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
-- exclude empty and nulls
and device_serial_number is not null
and device_serial_number not in ('UNKNOWN','','unknown','0123456789ABCDEF','100000000000000','0123456789')
-- THIS IS OPTIONAL depending on what type of users you want to get the connected accounts of
group by 1
),

calc_widevine_wifi_mac_address as( -- being treated as a composite key
select
  widevine_id,
  wifi_mac_address,
  'widevine+wifi_mac_address' as key,
  count(distinct CONCAT(x_owner_id, " ", x_owner_type)) as conn_account,
  array_agg(distinct CONCAT(x_owner_id, " ", x_owner_type)) as accounts_list
from `staging.fd_mdf_fingerprint_feature_flat` 
where true
and date(updated_time) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
-- exclude empty and nulls dexguard due to coverage issues
and widevine_id is not null
and widevine_id not in ('UNKNOWN','')
and wifi_mac_address is not null
and wifi_mac_address not in ('UNKNOWN','','02:00:00:00:00:00','00:00:00:00:00:00')
-- THIS IS OPTIONAL depending on what type of users you want to get the connected accounts of
group by 1,2
),

combine as(
select
  account_id,
  key,
  conn_account
from calc_dexguard, unnest(accounts_list) as account_id
union all
select
  account_id,
  key,
  conn_account
from calc_uniqueid, unnest(accounts_list) as account_id
union all
select
  account_id,
  key,
  conn_account
from calc_serial_number, unnest(accounts_list) as account_id
union all
select
  account_id,
  key,
  conn_account
from calc_widevine_wifi_mac_address, unnest(accounts_list) as account_id
),
customer_connected_devices AS (
SELECT 
TRIM(a.account_id, "customer") as customer_id,
max(conn_account) as conn_accounts
FROM combine a
left join `bi-gojek.reference.dd_customer` b  on TRIM(a.account_id, "customer") = cast(b.customer_id as STRING)
WHERE account_id like "%customer"
GROUP BY 1
),

driver_connected_devices AS (
SELECT 
TRIM(a.account_id, "driver") as driver_id,
max(conn_account) as conn_accounts
FROM combine a
left join `bi-gojek.reference.dd_driver` b  on TRIM(a.account_id, "driver") = cast(b.id as STRING)
WHERE account_id like "%driver"
GROUP BY 1
),
customer_info AS 
(
  SELECT customer_id, DATE(customer_created_at) AS customer_created_date,
  FROM `bi-gojek.reference.dd_customer`
),

customer_sanction_summary as 
(
  SELECT 
      target_id as customer_id, 
      sum(if(lower(action_name) IN ('suspenddriver', 'firedriver'),1,0)) sanction_cnt,
  FROM `bi-gojek.stream.fr_beast_sanction_execution_log`
  where DATE(event_timestamp) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
  AND target_type = 'customer'
  GROUP BY 1
),
  
driver_sanction_summary as 
(
  SELECT 
      target_id as driver_id, 
      sum(if(lower(action_name) IN ('suspenddriver', 'firedriver'),1,0)) sanction_cnt,
  FROM `bi-gojek.stream.fr_beast_sanction_execution_log`
  where DATE(event_timestamp) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
  AND target_type = 'driver'
  GROUP BY 1
),

driver_co_per_s2id AS (
  SELECT driver_id,
    booking_destination_s2id,
    count(distinct order_no) as co_per_s2id
  FROM `bi-gojek.access.sd_gofood_booking` AS t0
  WHERE DATE(t0._PARTITIONTIME) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
  GROUP BY 1, 2
),

drivermerchant_co_per_s2id AS (SELECT driver_id, merchant_id,
booking_destination_s2id,
count(distinct order_no) as co_per_s2id
FROM `bi-gojek.access.sd_gofood_booking` AS t0
WHERE DATE(t0._PARTITIONTIME) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
GROUP BY 1, 2, 3
),

receipt_data as 
(SELECT order_number,
   is_receipt_label, is_receipt,prediction_score, event_timestamp
   ,RANK() OVER(PARTITION BY order_number ORDER BY event_timestamp DESC) data_rank
from `bi-gojek.stream.fr_beast_go_food_fraud_receipt_multiclass_detection`
where DATE(event_timestamp) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
),


driver_level_features AS
( SELECT driver_id, 
    AVG(co_per_s2id) as d_co_per_s2id,
    COUNT(DISTINCT CASE WHEN DATE_DIFF(t0.booking_date, t2.customer_created_date, DAY) <= 7 THEN t2.customer_id ELSE NULL END) as d_unique_new_customer_cnt,
    SAFE_DIVIDE(COUNT(CASE WHEN DATE_DIFF(t0.booking_date, t2.customer_created_date, DAY) <= 7 THEN order_no ELSE NULL END), COUNT(*)) as d_new_cust_order_pct,
    SAFE_DIVIDE(COUNT(CASE WHEN voucher_id IS NOT NULL AND t0.status_id = 3 THEN 1 ELSE NULL END), COUNT(*)) AS d_voucher_pct,
    SAFE_DIVIDE(COUNT(CASE WHEN t3.sanction_cnt>0 THEN order_no ELSE NULL END), COUNT(*)) d_cmpltorders_from_suspicious_customer_pct, 
    count(distinct order_no) as d_bad_receipt_cnt
  FROM `bi-gojek.access.sd_gofood_booking` AS t0
    LEFT JOIN driver_co_per_s2id as t1 USING(driver_id) 
    LEFT JOIN customer_info AS t2 ON t2.customer_id = t0.customer_id
    LEFT JOIN customer_sanction_summary AS t3 ON cast(t0.customer_id as string) = t3.customer_id
    left join receipt_data b on t0.order_no=b.order_number
    WHERE DATE(t0._PARTITIONTIME) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
    AND t0.status_id = 3
    AND data_rank=1
    AND is_receipt_label = 'random'
    AND prediction_score >0.9
    GROUP BY 1
),


merchant_driver_fraud_signal AS 
(
  SELECT 
    t0.driver_id, t0.merchant_id,
    COUNT(DISTINCT CASE WHEN DATE_DIFF(t0.booking_date, t1.customer_created_date, DAY) <= 7 THEN t1.customer_id ELSE NULL END) as dm_unique_new_customer_cnt,
    SAFE_DIVIDE(COUNT(CASE WHEN DATE_DIFF(t0.booking_date, t1.customer_created_date, DAY) <= 7 THEN order_no ELSE NULL END), COUNT(*)) as dm_new_cust_order_pct,
    SAFE_DIVIDE(COUNT(CASE WHEN t2.sanction_cnt>0 THEN order_no ELSE NULL END), COUNT(*)) dm_cmpltorders_from_suspicious_customer_pct,
    SAFE_DIVIDE(COUNT(CASE WHEN payment_method_name = 'CASH' THEN 1 ELSE NULL END), COUNT(*)) AS dm_cash_order_pct,
    SAFE_DIVIDE(COUNT(CASE WHEN voucher_id IS NOT NULL THEN 1 ELSE NULL END), COUNT(*)) AS dm_voucher_pct,
    AVG(t3.co_per_s2id) as dm_avg_co_per_s2ID,
    count(distinct order_no) as dm_bad_receipt_cnt,
--     AVG(t5.conn_accounts) as dm_avg_cust_conn_accounts
  FROM `bi-gojek.access.sd_gofood_booking` AS t0
    LEFT JOIN customer_info AS t1 ON t1.customer_id = t0.customer_id
    LEFT JOIN customer_sanction_summary AS t2 ON cast(t0.customer_id as string) = t2.customer_id
    LEFT JOIN drivermerchant_co_per_s2id t3 ON t0.driver_id = t3.driver_id AND t0.merchant_id = t3.merchant_id
    left join receipt_data b on t0.order_no=b.order_number
    LEFT JOIN customer_connected_devices AS t5 ON t5.customer_id = cast(t0.customer_id as string)
    WHERE DATE(t0._PARTITIONTIME) between (select ref_date0 from date_list) and (select ref_date1 from date_list)
    AND t0.status_id = 3
    AND  data_rank=1
    AND is_receipt_label = 'random'
    AND prediction_score >0.9
    GROUP BY 1, 2
)

SELECT 
  distinct
  t0.*, 
  t1.* except(driver_id),
  t2.triangle_count,
--   t3.conn_accounts
FROM merchant_driver_fraud_signal AS t0 
LEFT JOIN driver_level_features AS t1 USING(driver_id)
LEFT JOIN `bi-gojek.stream.fr_beast_falcon_score` as t2 ON t2.resource_id = t0.driver_id
LEFT JOIN driver_connected_devices as t3 ON cast(t0.driver_id as string) = t3.driver_id
WHERE t2.version IN ('Falcon v0.2.0 (Triangle 1 Week)')

LIMIT 100000

In [ ]:
df3

,driver_id,merchant_id,dm_unique_new_customer_cnt,dm_new_cust_order_pct,dm_cmpltorders_from_suspicious_customer_pct,dm_cash_order_pct,dm_voucher_pct,dm_avg_co_per_s2ID,dm_bad_receipt_cnt,d_co_per_s2id,d_unique_new_customer_cnt,d_new_cust_order_pct,d_voucher_pct,d_cmpltorders_from_suspicious_customer_pct,d_bad_receipt_cnt,triangle_count
0,540232345,542874335,0,0.0,0.0,0.0,0.0,1.000,2,1.950000,0,0.000000,0.073171,0.0,41,4.0
1,540232345,543254804,0,0.0,0.0,1.0,0.0,1.000,1,1.950000,0,0.000000,0.073171,0.0,41,4.0
2,540232345,542840818,0,0.0,0.0,0.0,0.0,1.000,1,1.950000,0,0.000000,0.073171,0.0,41,5.0
3,540232345,543371358,0,0.0,0.0,0.0,0.0,1.000,1,1.950000,0,0.000000,0.073171,0.0,41,1.0
4,540232345,542880067,0,0.0,0.0,0.5,0.0,1.125,2,1.950000,0,0.000000,0.073171,0.0,41,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93738,541947560,542623113,0,0.0,0.0,0.5,0.0,1.000,2,2.338462,1,0.015385,0.200000,0.0,65,27.0
93739,541947560,542563467,0,0.0,0.0,1.0,0.0,1.000,1,2.338462,1,0.015385,0.200000,0.0,65,18.0
93740,541947560,542563467,0,0.0,0.0,1.0,0.0,1.000,1,2.338462,1,0.015385,0.200000,0.0,65,14.0
93741,541947560,542563467,0,0.0,0.0,1.0,0.0,1.000,1,2.338462,1,0.015385,0.200000,0.0,65,1.0


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

from google.colab import files
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


Defining functions

In [ ]:
def data_to_csv(data):
  data.to_csv("output.csv")
  files.download("output.csv")

In [ ]:
scaler = StandardScaler()
pca = PCA(n_components=0.835)
std_pca = make_pipeline(scaler, pca)

Data preparation

In [ ]:
df

,driver_id,merchant_id,dm_unique_new_customer_cnt,dm_new_cust_order_pct,dm_cmpltorders_from_suspicious_customer_pct,dm_cash_order_pct,dm_voucher_pct,dm_avg_co_per_s2ID,dm_bad_receipt_cnt,d_co_per_s2id,d_unique_new_customer_cnt,d_new_cust_order_pct,d_voucher_pct,d_cmpltorders_from_suspicious_customer_pct,d_bad_receipt_cnt,triangle_count


In [ ]:
df1 = df3.drop(columns=['d_cmpltorders_from_suspicious_customer_pct', 'dm_cmpltorders_from_suspicious_customer_pct'])
train_set = df1.dropna()
X_train_pure, X_test_pure = train_test_split(df2, test_size=0.33, random_state=42)
train_set = X_train_pure.drop(columns=['driver_id', 'merchant_id'])
test_set = X_test_pure.drop(columns=['driver_id', 'merchant_id'])

In [ ]:
train_set

,dm_unique_new_customer_cnt,dm_new_cust_order_pct,dm_cash_order_pct,dm_voucher_pct,dm_avg_co_per_s2ID,dm_bad_receipt_cnt,d_co_per_s2id,d_unique_new_customer_cnt,d_new_cust_order_pct,d_voucher_pct,d_bad_receipt_cnt,triangle_count
59428,0,0.000000,1.0,0.000000,1.00,1,2.142857,1,0.041667,0.083333,24,1.0
34957,0,0.000000,1.0,0.000000,1.00,1,1.047619,0,0.000000,0.111111,9,1.0
4264,1,0.333333,1.0,0.000000,1.80,3,6.181818,38,0.475610,0.524390,82,34.0
53791,1,0.200000,0.8,0.000000,2.20,5,3.047619,1,0.050000,0.150000,20,1.0
82114,1,1.000000,0.0,1.000000,1.00,1,1.243243,35,0.782609,1.000000,46,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6265,1,1.000000,0.0,1.000000,1.00,1,1.295455,47,0.888889,0.981481,54,4.0
54886,0,0.000000,1.0,1.000000,1.00,1,2.062500,0,0.000000,0.809524,21,1.0
76820,0,0.000000,1.0,0.166667,1.25,6,2.821429,5,0.128205,0.564103,39,6.0
860,0,0.000000,1.0,0.000000,1.00,1,2.516129,27,0.369863,0.589041,73,9.0


Standardization and dimensionality reduction

In [ ]:
X_train = pd.DataFrame(std_pca.fit_transform(train_set))
X_test = pd.DataFrame(std_pca.transform(test_set))

Percentiles based analysis data preparation

In [ ]:
train_quant = train_set.quantile([0.3, 0.4, 0.5, 0.75, 0.85, 0.9, 0.99])
train_mean = train_set.mean()
train_mean  = train_mean.to_frame()
train_mean = train_mean.transpose()
train_mean = train_mean.rename(index = {0.00: 'mean'})
train_std = train_set.std()
train_std = train_std.to_frame()
train_std = train_std.transpose()
train_std = train_std.rename(index = {0.00: 'std'})
t1 = train_quant.append(train_std)
t1 = t1.append(train_mean)

In [ ]:
data_to_csv(t1)

Outlier detection: 85-percentile and >7 features signal fraud

In [ ]:
_perc = train_set.quantile([0.5, 0.85])
_perc = _perc.rename(index = {0.5: 'perc50', 0.85: 'perc85'})
# test_set_withperc = test_set.append(_perc)
# test_set_withperc

In [ ]:
_perc

,dm_unique_new_customer_cnt,dm_new_cust_order_pct,dm_cash_order_pct,dm_voucher_pct,dm_avg_co_per_s2ID,dm_bad_receipt_cnt,d_co_per_s2id,d_unique_new_customer_cnt,d_new_cust_order_pct,d_voucher_pct,d_bad_receipt_cnt,triangle_count
perc50,0.0,0.0,1.0,0.0,1.000000,1.0,1.844444,2.0,0.083333,0.333333,23.0,4.0
perc85,1.0,1.0,1.0,1.0,1.333333,3.0,2.957143,22.0,0.675325,0.989796,62.0,17.0


In [ ]:
data_to_csv(_perc)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
def strike_cnt(row): 
  s =sum([row['dm_unique_new_customer_cnt'] >= test_set_withperc['dm_unique_new_customer_cnt'].loc['perc85'], \
      row['dm_new_cust_order_pct'] >= test_set_withperc['dm_new_cust_order_pct'].loc['perc85'], \
      row['dm_cash_order_pct'] >= test_set_withperc['dm_cash_order_pct'].loc['perc85'], \
      row['dm_voucher_pct'] >= 0.7, \
      row['dm_avg_co_per_s2ID'] >= test_set_withperc['dm_avg_co_per_s2ID'].loc['perc85'], \
      row['dm_bad_receipt_cnt'] >= test_set_withperc['dm_bad_receipt_cnt'].loc['perc85'], \
      row['d_co_per_s2id'] >= test_set_withperc['d_co_per_s2id'].loc['perc85'], \
      row['d_unique_new_customer_cnt'] >= test_set_withperc['d_unique_new_customer_cnt'].loc['perc85'], \
      row['d_new_cust_order_pct'] >= test_set_withperc['d_new_cust_order_pct'].loc['perc50'], \
      row['d_voucher_pct'] >= 0.7, \
      row['d_bad_receipt_cnt'] >= test_set_withperc['d_bad_receipt_cnt'].loc['perc85'], \
      row['triangle_count'] >= test_set_withperc['triangle_count'].loc['perc85']])
  return s


In [ ]:
test_set_pure = X_test_pure
test_set_pure['feat_strike'] = test_set_pure.apply(strike_cnt, axis =1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
test_set_pure.head()

,driver_id,merchant_id,dm_unique_new_customer_cnt,dm_new_cust_order_pct,dm_cash_order_pct,dm_voucher_pct,dm_avg_co_per_s2ID,dm_bad_receipt_cnt,d_co_per_s2id,d_unique_new_customer_cnt,d_new_cust_order_pct,d_voucher_pct,d_bad_receipt_cnt,triangle_count,feat_strike
75721,700295870,542547727,0,0.0,0.2,0.0,3.098361,5,3.194444,0,0.000000,0.000000,5,15.0,4
80184,540967803,542858196,0,0.0,1.0,0.0,1.000000,1,2.475248,1,0.166667,0.666667,6,1.0,2
19864,700786792,543314718,1,1.0,0.0,1.0,2.000000,1,1.512000,102,0.944444,0.981481,108,9.0,8
76699,700776179,542794064,0,0.0,1.0,0.0,1.000000,1,5.768293,0,0.000000,0.000000,5,1.0,3
92991,700504875,542953839,0,0.0,1.0,0.0,2.677419,4,4.217391,0,0.000000,0.125000,8,3.0,5


In [ ]:
outlier_from_perc = X_test_pure[(test_set_pure['feat_strike'] > 7)]
outlier_from_perc.shape

(1518, 15)

In [ ]:
data_to_csv(outlier_from_perc)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Fitting data to outlier detection model

Isolation forest

In [ ]:

clf = IsolationForest(max_samples=50000, random_state=0)
clf.fit(X_train) # train model
v_predict_arr = clf.predict(X_test) # get prediction on test
v_predict = pd.DataFrame(v_predict_arr) # convert prediction to dataframe
v_predict.columns = ["iso_for_outliers"] 
# X_test_pure["iso_for_outliers"] = v_predict
v_score_arr = clf.decision_function(X_test)
v_score = pd.DataFrame(v_score_arr)
v_predict["ano_score"] = v_score 
# X_test_pure["ano_score"] = v_score 

In [ ]:
X_test_pure.head()

In [ ]:
v_predict.head()

,iso_for_outliers,ano_score
0,1,0.158940
1,1,0.147696
2,1,0.078912
3,1,0.154443
4,1,0.098877


In [ ]:
v_predict[v_predict["ano_score"]<-0.15].describe()

,iso_for_outliers,ano_score
count,86.0,86.000000
mean,-1.0,-0.185223
std,0.0,0.028409
min,-1.0,-0.307883
25%,-1.0,-0.201921
50%,-1.0,-0.180503
75%,-1.0,-0.162947
max,-1.0,-0.150046


In [ ]:
to_validate = X_test_pure[(v_score_arr<-0.15)]
data_to_csv(to_validate)

Outlier mean with all features incorporated. This will be compared with backward elimination result

In [ ]:
outlier_mean = v_score[v_score < -0.1].mean()
print(outlier_mean)

0   -0.147598
dtype: float64


In [ ]:
cols = list(df_mod.columns)
cols[len(cols)-1]

'triangle_count'

Loop to try different n_component values

In [ ]:
def loop_pca(start, end, count):
  column_names = ["PCA n_components", "Outlier avg", "Number of outliers"]
  result = pd.DataFrame(columns = column_names)
  clf = IsolationForest(max_samples=50000, random_state=0)
  for n_comp in np.linspace(start,end,count):
    print("Trying with n_component: ", n_comp)
    scaler_loop = StandardScaler()
    pca_loop = PCA(n_components = n_comp)
    std_pca_loop = make_pipeline(scaler_loop, pca_loop)
    X_train_n = pd.DataFrame(std_pca_loop.fit_transform(train_set))
    X_test_n = pd.DataFrame(std_pca_loop.transform(test_set))
    print("Size of dataset", X_train_n.shape)
    clf.fit(X_train_n) # train model
    score_out = clf.decision_function(X_test_n)
    mean_out = score_out[score_out < -0.1].mean()
    count_out = len(score_out[score_out < -0.1])
    result = result.append({'PCA n_components' : n_comp , "Outlier avg" : mean_out, "Number of outliers" : count_out}, ignore_index = True)
  return result

In [ ]:
pca_loop_result = loop_pca(0.4, 0.999, 12)

Trying with n_component:  0.4
Size of dataset (67000, 2)
Trying with n_component:  0.45445454545454544
Size of dataset (67000, 2)
Trying with n_component:  0.508909090909091
Size of dataset (67000, 2)
Trying with n_component:  0.5633636363636364
Size of dataset (67000, 2)
Trying with n_component:  0.6178181818181818
Size of dataset (67000, 2)
Trying with n_component:  0.6722727272727272
Size of dataset (67000, 3)
Trying with n_component:  0.7267272727272727
Size of dataset (67000, 3)
Trying with n_component:  0.7811818181818182
Size of dataset (67000, 4)
Trying with n_component:  0.8356363636363636
Size of dataset (67000, 5)
Trying with n_component:  0.890090909090909
Size of dataset (67000, 6)
Trying with n_component:  0.9445454545454545
Size of dataset (67000, 8)
Trying with n_component:  0.999
Size of dataset (67000, 12)


In [ ]:
data_to_csv(pca_loop_result)

In [ ]:
outlier_mean_list

Backward elimination

In [ ]:
df_samp = df.iloc[:, 2:len(df.columns)]
x_test = df_samp.sample(50000)

In [ ]:
df_samp.head()

,dm_unique_new_customer_cnt,dm_new_cust_order_pct,dm_cmpltorders_from_suspicious_customer_pct,dm_cash_order_pct,dm_voucher_pct,dm_avg_co_per_s2ID,dm_bad_receipt_cnt,d_co_per_s2id,d_unique_new_customer_cnt,d_new_cust_order_pct,d_voucher_pct,d_cmpltorders_from_suspicious_customer_pct,d_bad_receipt_cnt,triangle_count
23526,2,1.0,0.0,0.0,1.0,1.000000,2,2.114537,262,0.907216,0.996564,0.0,291,2.0
41756,0,0.0,0.0,1.0,1.0,1.000000,1,6.278689,67,0.258555,0.653992,0.0,263,3.0
45125,0,0.0,0.0,1.0,0.0,1.000000,1,4.455882,0,0.000000,0.090909,0.0,11,1.0
28954,0,0.0,0.0,1.0,0.0,1.333333,1,5.538462,0,0.000000,0.166667,0.0,66,3.0
16293,1,1.0,0.0,0.0,1.0,1.000000,1,1.298246,25,0.806452,1.000000,0.0,31,7.0


In [ ]:
#Backward Elimination
df_mod = df2.drop(columns=['driver_id', 'merchant_id'])
df_cols = list(df_mod.columns)
column_names = ["Removed_feature", "Outlier avg"]
outlier_mean_list = pd.DataFrame(columns = column_names)
clf = IsolationForest(max_samples=50000, random_state=0)
# outlier_mean_list = []
for i in list(range(len(df_cols))):
    cols = list(df_mod.columns)
    feature2remove = cols[i]
    print("Removing feature", feature2remove, "at index", i)
    cols.remove(feature2remove)
    X_1_train = train_set[cols]
    X_1_test = test_set[cols]
    X_1_train = pd.DataFrame(std_pca.fit_transform(X_1_train))
    X_1_test = pd.DataFrame(std_pca.transform(X_1_test))
    # X_1_train = train_set.drop([feature2remove])
    # X_1_test = test_set.drop([feature2remove])
    # X_1_train = pd.DataFrame(std_pca.fit_transform(X_1_train))
    # X_1_test = pd.DataFrame(std_pca.transform(X_1_test))
    print("Size of dataset is: ", X_1_test.shape)
    output = clf.fit(X_1_train)
    out_score = output.decision_function(X_1_test)
    outlier_mean = out_score[out_score < -0.1].mean()
    outlier_count = len(out_score[out_score < -0.1])
    outlier_mean_list = outlier_mean_list.append({'Removed_feature' : feature2remove , "Outlier avg" : outlier_mean, "Number of outliers" : outlier_count}, ignore_index = True)     
  

Removing feature dm_unique_new_customer_cnt at index 0
Size of dataset is:  (33000, 5)
Removing feature dm_new_cust_order_pct at index 1
Size of dataset is:  (33000, 5)
Removing feature dm_cash_order_pct at index 2
Size of dataset is:  (33000, 5)
Removing feature dm_voucher_pct at index 3
Size of dataset is:  (33000, 5)
Removing feature dm_avg_co_per_s2ID at index 4
Size of dataset is:  (33000, 5)
Removing feature dm_bad_receipt_cnt at index 5
Size of dataset is:  (33000, 5)
Removing feature d_co_per_s2id at index 6
Size of dataset is:  (33000, 5)
Removing feature d_unique_new_customer_cnt at index 7
Size of dataset is:  (33000, 5)
Removing feature d_new_cust_order_pct at index 8
Size of dataset is:  (33000, 5)
Removing feature d_voucher_pct at index 9
Size of dataset is:  (33000, 5)
Removing feature d_bad_receipt_cnt at index 10
Size of dataset is:  (33000, 5)
Removing feature triangle_count at index 11
Size of dataset is:  (33000, 4)


In [ ]:
outlier_mean_list

,Removed_feature,Outlier avg,Number of outliers
0,dm_unique_new_customer_cnt,-0.156292,257.0
1,dm_new_cust_order_pct,-0.157472,269.0
2,dm_cash_order_pct,-0.155041,285.0
3,dm_voucher_pct,-0.161772,260.0
4,dm_avg_co_per_s2ID,-0.153258,270.0
5,dm_bad_receipt_cnt,-0.157380,261.0
6,d_co_per_s2id,-0.157745,290.0
7,d_unique_new_customer_cnt,-0.153454,268.0
8,d_new_cust_order_pct,-0.158333,269.0
9,d_voucher_pct,-0.159431,274.0


In [ ]:
data_to_csv(outlier_mean_list)

In [ ]:
train_set

,dm_unique_new_customer_cnt,dm_new_cust_order_pct,dm_cash_order_pct,dm_voucher_pct,dm_avg_co_per_s2ID,dm_bad_receipt_cnt,d_co_per_s2id,d_unique_new_customer_cnt,d_new_cust_order_pct,d_voucher_pct,d_bad_receipt_cnt,triangle_count
59428,1,1.000000,0.000000,1.000000,1.00,1,1.507937,67,0.807229,1.000000,83,13.0
34957,0,0.000000,1.000000,0.000000,2.75,1,2.938776,0,0.000000,0.500000,2,7.0
4264,0,0.000000,1.000000,1.000000,1.80,5,7.491803,77,0.534247,0.876712,146,3.0
53791,0,0.000000,0.000000,0.000000,1.00,1,4.491525,27,0.421875,0.890625,64,1.0
82114,2,0.666667,0.333333,1.000000,1.00,3,1.700730,127,0.830065,0.993464,153,23.0
...,...,...,...,...,...,...,...,...,...,...,...,...
6265,0,0.000000,1.000000,0.333333,1.25,3,6.440000,24,0.303797,0.658228,79,4.0
54886,0,0.000000,1.000000,1.000000,1.00,1,1.708738,55,0.785714,1.000000,70,2.0
76820,1,1.000000,0.000000,1.000000,1.00,1,1.489362,108,0.782609,1.000000,138,11.0
860,1,1.000000,0.000000,1.000000,1.00,1,1.518367,178,0.967391,0.994565,184,4.0


In [ ]:
outlier_mean_list

,Removed_feature,Outlier avg,Number of outliers
0,dm_unique_new_customer_cnt,-0.15158,273.0
1,dm_new_cust_order_pct,-0.15158,273.0
2,dm_cash_order_pct,-0.15158,273.0
3,dm_voucher_pct,-0.15158,273.0
4,dm_avg_co_per_s2ID,-0.15158,273.0
5,dm_bad_receipt_cnt,-0.15158,273.0
6,d_co_per_s2id,-0.15158,273.0
7,d_unique_new_customer_cnt,-0.15158,273.0
8,d_new_cust_order_pct,-0.15158,273.0
9,d_voucher_pct,-0.15158,273.0
